# Timeseries Spatial Average Demo

This notebook will demonstrate how to call the code to produce timeseries DataSubsets and plots for several different forms of input data requiring different spatial calculations, namely the following:
1. A box (defined by cell coordinates at the corners of the box)
2. A polygon (defined as a shapely Polygon object)
3. Multiple polygons (defined as a shapely MultiPolygon object).

Using spatial averaging will provide a timeseries of the mean data values for the given 2D shape, defaulting to ground level for altitude. The data within the shape is extracted and then averaged, to provide new data points for the timeseries.

In [ ]:
import os
from shapely.geometry import Polygon, MultiPolygon

import cap_sample_data
from clean_air.visualise import dataset_renderer

## Load the data

Define your input data - here we have used some gridded model data. 
Data can be passed to the Timeseries calculator as either a filepath or a DataSubset object.  
Given that the calculator itself makes the conversion to the DataSubset object, we will use a simple filepath.

In [ ]:
SAMPLEDIR = cap_sample_data.path
raw_data_filepath = os.path.join(SAMPLEDIR, "model_full", "aqum_hourly_o3_20200520.nc")

## Scenario 1: Plotting a box

Load the data through Timeseries with spatial average specified.  
Here we have used arbitrary values for coords to use as the corners of the box (in the form (xmin, ymin, xmax, ymax)), and also specified 'box' as the shape we want to use for data processing.

In [ ]:
box_data = dataset_renderer.TimeSeries(raw_data_filepath).spatial_average(shape='box', coords=[0, 0, 6000, 8000])

We can now plot our Timeseries object.

In [ ]:
box_df = dataset_renderer.Renderer(box_data).render()
box_df.hvplot()

## Scenario 2: Plotting a polygon

Here we have used arbitrary coordinates as the corners of a very simple Polygon object.

In [ ]:
shape = Polygon([(0, 0), (1000, 1000), (1000, 0)])

Now we can pass this shape to the Timeseries calculator to load a subset, as in the previous scenario.

In [ ]:
shape_data = dataset_renderer.TimeSeries(raw_data_filepath).spatial_average(shape)

And now we can plot the object.


In [ ]:
shape_df = dataset_renderer.Renderer(shape_data).render()
shape_df.hvplot()

## Scenario 3: Plotting multiple polygons as side-by-side plots 

In this scenario we have input in the form of a shapely MultiPolygon object representing a list of Polygons.  There are a lot of rules about how we define these objects, please see this link for further information:
https://shapely.readthedocs.io/en/stable/manual.html#MultiPolygon

Here we have defined two triangles which do not intersect each other and put them in a list to create a MultiPolygon object.

In [ ]:
poly_one = Polygon([(0, 0), (100000, 100000), (100000, 0)])
poly_two = Polygon([(-100000, -100000), (-90000, -90000), (-90000, -10000)])
shapes = MultiPolygon([poly_one, poly_two])

We can use this object in the same way that we do single Polygon objects to extract a list of DataSubsets, and then find the spatial average for each.

In [ ]:
shapes_data = dataset_renderer.TimeSeries(raw_data_filepath).spatial_average(shapes)

Once again we can use this to call the renderer. This will produce a pandas dataframe with additional columns, one for each polygon. These will then be automatically plotted as overlaid lines.

In [ ]:
shapes_df = dataset_renderer.Renderer(shapes_data).render()
shapes_df.hvplot(title="Mass Concentration of Ozone in Air Timeseries", value_label="Ozone in ug m-3")

We can also produce separate subplots, simply using `subplots=True` (more information is available __[here](https://hvplot.holoviz.org/user_guide/Subplots.html)__).  

In [ ]:
shapes_df.hvplot(subplots=True, width=500, height=400, value_label="Ozone in ug m-3")